In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .config("spark.driver.bindAddress", "127.0.0.1") \
    .config("spark.driver.host", "127.0.0.1") \
    .appName("YourAppName") \
    .getOrCreate()



25/03/12 19:35:37 WARN Utils: Your hostname, tanvi-linux resolves to a loopback address: 127.0.1.1; using 192.168.1.74 instead (on interface wlo1)
25/03/12 19:35:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/12 19:35:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [67]:
from datetime import datetime

print(datetime.now())
spark.sparkContext

2025-03-12 20:05:45.001094


<SparkContext master=local[*] appName=YourAppName>

In [ ]:
# import json 

# with open("/home/tanvi/Documents/GitRepo/automatic-system/etl/data/weather.json", "r") as opfile:
#     data = json.load(opfile)

# print(data)

# with open("/home/tanvi/Documents/GitRepo/automatic-system/etl/data/weather_new.json", "w") as f:
#     for record in data:
#         f.write(json.dumps(record) + "\n")  # ✅ Writes each object on a new line.


{'location': {'name': 'Piscataway', 'region': 'New Jersey', 'country': 'USA', 'xxx': 'aaa', 'lat': 40.5527, 'lon': -74.4582, 'tz_id': 'America/New_York', 'localtime_epoch': 1740332071, 'localtime': '2025-02-23 12:34'}, 'current': {'last_updated_epoch': 1740331800, 'last_updated': '2025-02-23 12:30', 'temp_c': 6.7, 'temp_f': 44.1, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 9.2, 'wind_kph': 14.8, 'wind_degree': 283, 'wind_dir': 'WNW', 'pressure_mb': 1019.0, 'pressure_in': 30.1, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 40, 'cloud': 0, 'feelslike_c': 3.9, 'feelslike_f': 39.0, 'windchill_c': 0.1, 'windchill_f': 32.3, 'heatindex_c': 3.3, 'heatindex_f': 38.0, 'dewpoint_c': -5.6, 'dewpoint_f': 21.9, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 3.2, 'gust_mph': 11.1, 'gust_kph': 17.9}, 'forecast': {'forecastday': [{'date': '2025-02-23', 'date_epoch': 1740268800, 'day': {'maxtemp_c': 8.3, 'maxtemp_f': 46.9, '

In [79]:
df = spark.read.json("/home/tanvi/Documents/GitRepo/automatic-system/etl/data/weather/weather_20250312.json")

In [84]:
df.printSchema()
df.show()

newdf = df.select("location")
newdf.show()

root
 |-- location: struct (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- lat: double (nullable = true)
 |    |-- localtime: string (nullable = true)
 |    |-- localtime_epoch: long (nullable = true)
 |    |-- lon: double (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- region: string (nullable = true)
 |    |-- tz_id: string (nullable = true)

+--------------------+
|            location|
+--------------------+
|{USA, 40.55149841...|
+--------------------+

+--------------------+
|            location|
+--------------------+
|{USA, 40.55149841...|
+--------------------+



In [26]:
!ls -ltr /home/tanvi/Documents/GitRepo/automatic-system/etl/data/weather_deindent.json
#!chmod 777 /home/tanvi/Documents/GitRepo/automatic-system/etl/data/weather.json
#cat /home/tanvi/Documents/GitRepo/automatic-system/etl/data/weather.json | python -m json.tools

-rwxrwxrwx 1 tanvi tanvi 2971 Mar 12 19:29 /home/tanvi/Documents/GitRepo/automatic-system/etl/data/weather_deindent.json


In [27]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *  # Optional: to use SQL functions like col(), lit(), etc.

df.show()

# df.createOrReplaceTempView("weather_json")
# query = "SELECT * FROM weather_json"
# result = spark.sql(query)


+---+--------------------+
| id|           json_data|
+---+--------------------+
|  1|{"name": "John", ...|
|  2|{"name": "Jane", ...|
+---+--------------------+



In [35]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import get_json_object, col

result = df.select( "location"
    # get_json_object("location", "$.name").alias("name") 
    )

result.show(truncate=False)

# newdf = df.select("location", "current")

result.show(5)

+-----------------------------------------------------------------------------------------------------+
|location                                                                                             |
+-----------------------------------------------------------------------------------------------------+
|{USA, 40.5527, 2025-02-23 12:34, 1740332071, -74.4582, Piscataway, New Jersey, America/New_York, aaa}|
+-----------------------------------------------------------------------------------------------------+

+--------------------+
|            location|
+--------------------+
|{USA, 40.5527, 20...|
+--------------------+



In [36]:

data = [
    ("1", """{"name": "John", "age": 30, "address": {"city": "New York", "zip": "10001"}, "phones": [{"type": "home", "number": "212-555-1234"}, {"type": "work", "number": "646-555-5678"}]}"""),
    ("2", """{"name": "Jane", "age": 25, "address": {"city": "Boston", "zip": "02108"}, "phones": [{"type": "mobile", "number": "617-555-4321"}]}""")
]

samdf = spark.createDataFrame(data, ["id", "json_data"])

# Extract various fields
result = samdf.select(
    "id",
    get_json_object("json_data", "$.name").alias("name"),
    get_json_object("json_data", "$.age").alias("age"),
    get_json_object("json_data", "$.address.city").alias("city"),
    get_json_object("json_data", "$.phones[0].number").alias("first_phone")
)

# result.show(truncate=False)
samdf.show()

+---+--------------------+
| id|           json_data|
+---+--------------------+
|  1|{"name": "John", ...|
|  2|{"name": "Jane", ...|
+---+--------------------+

